# Perceptron de Rosenblatt  optimizado

## Algoritmo de Rosenblatt

### Pseudocódigo del Algoritmo de Rosenblatt

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/PeceptronEntrenamiento.jpg' />

### Código en Python del Algoritmo de Rosenblatt

Código en Python que grafica como va evolucionando la recta que separa a los positivos de los negativos

In [ ]:
# conexion al Google Drive
from google.colab import drive
drive.mount('/content/.drive')
!mkdir -p "/content/.drive/My Drive/DMA"
!mkdir -p "/content/buckets"
!ln -s "/content/.drive/My Drive/DMA" /content/buckets/b1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import time

In [ ]:
# Ejemplo, usted deberá generar sus propios puntos en el plano
#  los que mandatoriamente deben ser separables por una recta

registros = [ [0.7, 1.3], [2.0, 1.1], [1.0, 1.9],
            [3.0, 1.0], [1.5, 2.1] ]
clases = [0,0,0,1,1]


In [ ]:
# Paso las listas a numpy
X = np.array(registros)
Y = np.array(clases)

In [ ]:
# Tamano datos
X_row = X.shape[0]
X_col = X.shape[1]

In [ ]:
class perceptron_plot:
    def __init__(self, X, Y, delay) -> None:
        self.X = X
        self.Y = Y
        self.delay = delay
        x1_min = np.min(X[:,0])
        x2_min = np.min(X[:,1])
        x1_max = np.max(X[:,0])
        x2_max = np.max(X[:,1])
        self.x1_min = x1_min - 0.75*(x1_max - x1_min)
        self.x1_max = x1_max + 0.75*(x1_max - x1_min)
        self.x2_min = x2_min - 0.75*(x2_max - x2_min)
        self.x2_max = x2_max + 0.75*(x2_max - x2_min)
        self.fig = plt.figure(figsize = (10,8))
        self.ax = self.fig.subplots()
        self.ax.set_xlim(self.x1_min, self.x1_max, auto=False)
        self.ax.set_ylim(self.x2_min, self.x2_max, auto=False)

    def graficar(self, W, x0, epoch, fila) -> None:
        display.clear_output(wait =True)
        plt.cla()
        #self.ax = self.fig.subplots()

        self.ax.set_xlim(self.x1_min, self.x1_max)
        self.ax.set_ylim(self.x2_min, self.x2_max)
        plt.title( 'epoch ' + str(epoch) + '  reg ' + str(fila) )
        # ploteo puntos positivos
        self.ax.plot(self.X[self.Y==1,0], self.X[self.Y==1,1], 'o', color="green", markersize=10)
        # ploteo puntos negativos
        self.ax.plot(self.X[self.Y==0,0], self.X[self.Y==0,1], 'o', color="blue", markersize=10)

        # Sobreploteo el punto que no coincidio
        if(fila>=0):
            self.ax.plot(self.X[fila,0], self.X[fila,1], 'o',
                         color= ('green' if self.Y[fila]==1 else 'blue'),
                         markersize= 12, markeredgecolor= 'red')

        #dibujo le recta
        vx2_min = -(W[0]*self.x1_min + x0)/W[1]
        vx2_max = -(W[0]*self.x1_max + x0)/W[1]

        self.ax.plot([self.x1_min, self.x1_max],
                     [vx2_min, vx2_max],
                     linewidth = 2,
                     color = 'red',
                     alpha = 0.5)

        display.display(plt.gcf())
        #plt.cla()
        time.sleep(self.delay)


    def graficarVarias(self, W, x0, epoch, fila) -> None:
        display.clear_output(wait =True)
        plt.cla()
        #self.ax = self.fig.subplots()

        self.ax.set_xlim(self.x1_min, self.x1_max)
        self.ax.set_ylim(self.x2_min, self.x2_max)
        plt.title( 'epoch ' + str(epoch) + '  reg ' + str(fila))
        # ploteo puntos positivos
        self.ax.plot(self.X[self.Y==1,0], self.X[self.Y==1,1], 'o', color="green", markersize=10)
        # ploteo puntos negativos
        self.ax.plot(self.X[self.Y==-1,0], self.X[self.Y==-1,1], 'o', color="blue", markersize=10)
        self.ax.plot(self.X[self.Y==0,0], self.X[self.Y==0,1], 'o', color="blue", markersize=10)

        # Sobreploteo el punto que no coincidio
        if(fila>=0):
            self.ax.plot(self.X[fila,0], self.X[fila,1], 'o',
                         color= ('green' if self.Y[fila]==1 else 'blue'),
                         markersize= 12, markeredgecolor= 'red')

        # dibujo las rectas
        for i in range(len(x0)):
            #vx2_min = -(W[0,i]*self.x1_min + x0[i])/W[1,i]
            #vx2_max = -(W[0,i]*self.x1_max + x0[i])/W[1,i]
            vx2_min = -(W[i,0]*self.x1_min + x0[i])/W[i,1]
            vx2_max = -(W[i,0]*self.x1_max + x0[i])/W[i,1]

            self.ax.plot([self.x1_min, self.x1_max],
                         [vx2_min, vx2_max],
                         linewidth = 2,
                         color = 'red',
                         alpha = 0.5)

        display.display(plt.gcf())
        #plt.cla()
        time.sleep(self.delay)


In [ ]:
# Incializo la recta azarosamente

# seteo de la semilla aleatoria
np.random.seed(102191) # mi querida random seed para que las corridas sean reproducibles

In [ ]:
# inicializo < x0, W >
x0 = np.random.uniform(-0.5, 0.5)
W = np.array( np.random.uniform(-0.5, 0.5, size=X_col))

In [ ]:
# Limite de lo que estoy dispuesto a trabajar
epoch_limit = 500    # para terminar si no converge

In [ ]:
# controla la velocidad de convergencia
learning_rate = 0.01

In [ ]:
# controla cada cuanto se grafica un epoch
demora_impresion = 1.0  # segundos

In [ ]:
# inicializaciones del buble
modificados = 1      # lo debo poner algo distinto a 0 la primera vez
epoch = 0

In [ ]:
# el bucle principal del algoritmo de Rosenblatt

grafico = perceptron_plot(X=X, Y=Y, delay=demora_impresion)

# bucle principal del algoritmo de Rosenblatt
# continuo mientras en la iteración anterior modifique algo  y NO llegué al límite de epochs
while (modificados and (epoch < epoch_limit)):
    epoch += 1
    modificados = 0  # lo seteo en cero

    # recorro siempre TODOS los registros de entrada
    for fila in range(X_row):
        # fila es el registro actual
        x = X[fila,]
        clase = Y[fila]
        # calculo el estimulo suma, producto interno
        estimulo = x0*1 + W[0]*x[0] + W[1]*x[1]

        y = (estimulo > 0 )

        # solo si la prediccion es incorrecta  actualizo  < x0, W >
        gradiente = clase - y
        if(gradiente != 0):
            modificados += 1  # encontre un registro que esta mal clasificado
            # actualizo W y x0
            W[0] = W[0] + learning_rate * gradiente * x[0]
            W[1] = W[1] + learning_rate * gradiente * x[1]
            x0 =   x0   + learning_rate * gradiente * 1
            grafico.graficar(W, x0, epoch, fila) # actualizo grafico


In [ ]:
grafico = perceptron_plot(X, Y, 0)
grafico.graficar(W, x0, epoch, -1)

In [ ]:
# la cantidad de epochs necesarias hasta encontrar una solucion
print("Para converger hicieron falta epochs=",epoch)

In [ ]:
# el vector solución W
print( "La solucion es W = ", W)

In [ ]:
# El  "bias"  o componente  x0
print( "La solucion es en la dimension artificial x0 = ", x0)